In [1]:
from PyPDF2 import PdfReader
import os


file_path = '../data/PN DE INNOVACIÓN AGRARIA-E3.pdf'

def read_pdf(file_path: str):
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PdfReader(file)
            num_pages = len(pdf_reader.pages)
            text = ''
            for page_num in range(num_pages):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
            return text
    except FileNotFoundError:
        return f'PDF file not found in: {file_path}'
    except Exception as e:
        return f'An error occured while reading: {e}'

pdf_text = read_pdf(file_path)
len(pdf_text)

204153

In [7]:
from haystack.nodes import PreProcessor, PromptModel, PromptTemplate, PromptNode
from haystack import Document

doc = Document(
    content = pdf_text,
    meta = {"pdf_path": file_path}
)

docs = [doc]

processor = PreProcessor(
  clean_empty_lines=True,
  clean_whitespace=True,
  clean_header_footer=True,
  split_by='word',
  split_length=1000,
  split_overlap=50,
  split_respect_sentence_boundary=True,
  language='sp',
)

preprocessed_docs = processor.process(docs)
preprocessed_docs

[nltk_data] Downloading package punkt to /Users/Eber/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 13.11docs/s]


[<Document: {'content': '\n\nPOLÍTICA NACIONAL DE\nINNOVACIÓN AGRARIA\n\nTERCER ENTREGABLE\n\nOBJETIVOS PRIORITARIOS,  INDICADORES Y LINEAMIENTOS DE\nLA POLÍTICA NACIONAL DE INNOVACIÓN AGRARIA\n\nAgosto  de 2021\n\n2\n\nÍndice\n\n1. Proceso de elaboración de los objetos prioritarios, indicadores y lineamientos  ................................ ............................  3\n2. Definición de Objetivos Prioritarios  ................................ ................................ ................................ ................................ .... 4\n3. Elaboración de lineamientos  ................................ ................................ ................................ ................................ .............  15\n4. Elaboración de los indicadores  ................................ ................................ ................................ ................................ .........  39\n5. Referencias ................................ ...........................

In [9]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)
document_store.write_documents(preprocessed_docs)

Updating BM25 representation...: 100%|██████████| 34/34 [00:00<00:00, 3196.38 docs/s]


In [10]:
from haystack import Pipeline
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store, top_k=2)
retriever

In [24]:
from dotenv import load_dotenv

load_dotenv()
hf_api_key = os.environ.get("HF_TOKEN")
debug_mode = os.environ.get("DEBUG")

qa_template = PromptTemplate(
  """
  You are an assistant that answers exclusively in Spanish. Using only the information contained in the context, answer only the question asked without adding suggestions of possible questions. Cannot be deduced from the context, reply: "\I don't know because it isn't relevant to the Context.\"

  Context: {join(documents)};
  Question: {query}
  """
)

prompt_node = PromptNode(
  model_name_or_path = 'mistralai/Mixtral-8x7B-Instruct-v0.1',
  api_key = hf_api_key,
  default_prompt_template = qa_template,
  max_length = 500,
  model_kwargs = {'model_max_length': 5000}
)
prompt_node

In [25]:
rag_pipeline = Pipeline()
rag_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
rag_pipeline.add_node(component=prompt_node, name="PromptNode", inputs=["Retriever"])

from pprint import pprint
print_answer = lambda out: pprint(out["results"][0].strip())

In [26]:
print_answer(rag_pipeline.run(
  query="Lista objetivos prioritarios y sus indicadores en el siguiente formato \
 '[{código de objetivo prioritario: nombre de objetivo prioritario, indicadores: [indicador 1, indicador 2]}]'"))

The prompt has been truncated from 4605 tokens to 4500 tokens so that the prompt length and answer length (500 tokens) fit within the max token limit (5000 tokens). Shorten the prompt to prevent it from being cut off.


('ía de elaboración de indicadores de política pública,\n'
 'donde se establece que los indicadores deben ser específicos, relevantes, '
 'medibles, realizables y temporales\n'
 '(CEPLAN, 2021).\n'
 '\n'
 'Tabla N° 4. Matriz de Indicadores de la PNIA\n'
 '\n'
 'Objetivos Prioritarios  Indicadores\n'
 'OP.1. Garantizar la sostenibilidad de la gobernanza de la innovación '
 'agraria.  1.1. Intensidad de la innovación en el sector agrario.\n'
 '\n'
 '1.2. Porcentaje de empresas agrarias que acceden a beneficios tributarios '
 'por inversión de I+D+i con actores del SNIA en temas agrarios.\n'
 '\n'
 '1.3. Porcentaje de agendas implementadas con actores del SNIA que '
 'cofinancian investigación e innovación agraria.\n'
 '\n'
 'OP.2. Incrementar la generación de conocimientos de innovación agraria para '
 'los productores agrarios.  2.1. Tasa de publicaciones científicas agrarias '
 'publicadas en revisas indexadas y con autoría peruana por millón de '
 'habitantes.\n'
 '\n'
 '2.2. Tasa de 